Import necessary packages

In [1]:
import h5py    
import numpy as np    
import pandas as pd
import time,datetime,matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from dateutil import tz
import pytz
import datetime as dt
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import csv
import re

In [2]:
#import stick file (change as needed)
data= pd.read_csv('2022.07.27_UMR.csv')

df = pd.DataFrame(data=data).set_index('time')

In [3]:
#sum primary ions
PI = df['mz18']*np.sqrt(100/18)+df['mz36']*np.sqrt(100/36)+df['mz35']*np.sqrt(100/35)

# calculate the DCPS, NDCPS
# a note on DCPS- stands for duty cycle corrected counts per second. Accounts for the fact that
# ions of different masses take different amounts of time to reach the detector
masslist = list(range(1,707))
for i in df.columns.to_list():
    masslist.append(re.findall(r'\b\d+\b', i))
masslist_array=np.array(masslist[0:705])
masslist=masslist[0:705]
DCPS = df.mul(np.sqrt(100/masslist_array), axis=1)
NDCPS = DCPS.apply(lambda x: x*1e6/PI)


In [ ]:
#define plotting function
def Cps_plot(data, units, title):
    ax = plt.axes()
    plt.plot(data)
    # naming the x axis
    plt.xlabel('datetime')
    # naming the y axis
    plt.ylabel(units)
    # giving a title to my graph
    plt.title(title)
    plt.legend(data.columns, loc='center left', bbox_to_anchor=(1, 0.5), ncol=10)
   # formatter = mdates.DateFormatter('%m/%d %T %Z')#, tz=NDCPS.index.tz)
    #plt.gca().xaxis.set_major_formatter(formatter)
    ax.xaxis.set_major_locator(plt.MaxNLocator(4))
    plt.setp(plt.gca().xaxis.get_majorticklabels(),
         'rotation', 90)
    # function to show the plot
    plt.show()

#plot full experiment
Cps_plot(NDCPS, 'NDCPS', 'NDCPS v time')

In [ ]:
#remove CID scan
NDCPS_2 = NDCPS

# subset for experiment time
BGStart = '2022-07-27 10:00'
BGEnd = '2022-07-27 10:10'
SigAvgStart = '2022-07-27 10:44'
SigAvgEnd = '2022-07-27 13:00'

expt = NDCPS_2[((NDCPS_2.index >= SigAvgStart)
      &(NDCPS_2.index <= SigAvgEnd))]
bg= NDCPS_2[((NDCPS_2.index >= BGStart)
      &(NDCPS_2.index <= BGEnd))]

In [ ]:
#plot experiment data with all mz
Cps_plot(expt, 'NDCPS', 'NDCPS v time')

In [ ]:
#dilution correct with ACN
ACN = expt['mz59']
frac_remaining = ACN/ACN[1]

expt_corrected = expt.mul(1/frac_remaining[1], axis=1)

In [ ]:
#t-test to identify all compounds with a statistically significant change in signal
from scipy import stats

t_test = stats.ttest_ind(bg, expt_corrected)
t_test_df= pd.DataFrame(t_test).transpose()


#if p-value is more than 0.05, set value to N/A
for i in range(len(t_test_df)):
    if t_test_df[1][i] > 0.05:
        t_test_df[1][i] = np.nan

t_test_df=t_test_df.transpose()
t_test_df

In [ ]:
# background correct 
bg_avg = bg.mean(axis=0)
expt_bg_corrected = expt_corrected - bg_avg

#append p-values to dataset 
t_test_df.columns = expt_bg_corrected.columns
expt_bg_corrected = pd.concat([expt_bg_corrected, t_test_df], axis=0)

# get rid of columns with N/A p-values
expt_bg_corrected = expt_bg_corrected.dropna(axis='columns')

expt_bg_corrected = expt_bg_corrected.drop(axis=0, labels=[0,1])


In [ ]:
expt_final = expt_bg_corrected

In [ ]:
df_products= pd.DataFrame([expt_final['mz186'], expt_final['mz172']]).transpose()

In [ ]:
# plot limonene decay
Cps_plot(pd.DataFrame(expt_final['mz154']), 'NDCPS', 'limonene decay')

In [ ]:
# plot specific products
# 186=limal/limbco, 200= no assigned structure, 202=limononic, 204=limonic/ketolimononic, 156=limket
df_products= pd.DataFrame([expt_final['mz186'], expt_final['mz200'], expt_final['mz202'], expt_final['mz204'], expt_final['mz156']]).transpose()
Cps_plot(df_products, 'NDCPS', 'limonene products')


In [ ]:
Cps_plot(expt_final, 'NDCPS', 'limonene products')

In [ ]:
#drop primary ions

expt_inc = expt_final.drop(['mz18', 'mz35'], axis=1)


#create dataframe with only increasing species
expt_inc =expt_inc.loc[:, expt_inc.loc['2022-07-27 10:44:31']<expt_inc.loc['2022-07-27 12:59:31']]

# list of statistically significant increasing species between 50 and 300 mass
print(expt_inc.loc[:, 'mz51' :'mz300'].columns)

#plot statistically significant increasing species between 50 and 300 mass
Cps_plot(expt_inc.loc[:, 'mz51' :'mz300'], 'NDCPS', 'increasing')


#plot 10 masses with largest increase 
maximums = pd.DataFrame(expt_inc.drop(columns='seconds').max(axis = 0))
minimums = pd.DataFrame(expt_inc.drop(columns='seconds').min(axis = 0))
change =maximums-minimums
change.columns = ['change']
top10 = change.nlargest(10,'change')
top_10_df = expt_inc[top10.index]

Cps_plot(top_10_df, 'NDCPS', 'top 10')

In [ ]:
#plot 10 masses with largest increase in log scale
maximums = pd.DataFrame(expt_inc.drop(columns='seconds').max(axis = 0))
minimums = pd.DataFrame(expt_inc.drop(columns='seconds').min(axis = 0))
change =maximums-minimums
change.columns = ['change']
top10 = change.nlargest(10,'change')
top_10_df = expt_inc[top10.index]

Cps_plot(np.log(top_10_df), 'log(NDCPS)', 'top 10')

In [ ]:
#plot 10 masses with largest change 
maximums = pd.DataFrame(expt_final.drop(columns='seconds').max(axis = 0))
minimums = pd.DataFrame(expt_final.drop(columns='seconds').min(axis = 0))
change =maximums-minimums
change.columns = ['change']
top10 = change.nlargest(10,'change')
top_10_df = expt_final[top10.index]

Cps_plot(top_10_df, 'NDCPS', 'top 10')

In [ ]:
#plot dProduct v. dLimonene for top 10

plt.plot(expt_final['mz154'][1]-expt_final['mz154'], expt_corrected['mz186'], label='mz186')
plt.plot(expt_final['mz154'][1]-expt_final['mz154'], expt_corrected['mz202'], label='mz202')
plt.plot(expt_final['mz154'][1]-expt_final['mz154'], expt_corrected['mz188'], label='mz188')
plt.plot(expt_final['mz154'][1]-expt_final['mz154'], expt_corrected['mz92'], label='mz92')
plt.plot(expt_final['mz154'][1]-expt_final['mz154'], expt_corrected['mz187'], label='mz187')
plt.plot(expt_final['mz154'][1]-expt_final['mz154'], expt_corrected['mz169'], label='mz169')
plt.plot(expt_final['mz154'][1]-expt_final['mz154'], expt_corrected['mz185'], label='mz185')
plt.plot(expt_final['mz154'][1]-expt_final['mz154'], expt_corrected['mz233'], label='mz233')
plt.plot(expt_final['mz154'][1]-expt_final['mz154'], expt_corrected['mz174'], label='mz174')
plt.plot(expt_final['mz154'][1]-expt_final['mz154'], expt_corrected['mz204'], label='mz204')




plt.title ('product yield')
plt.ylabel('delta product (NDCPS)')
plt.xlabel('delta LIMONENE (NDCPS)')
plt.legend()